In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from IPython.display import display

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=1, keepdims=True)


file_name = 'mnist_test.csv'
df = pd.read_csv(file_name)

labels = df['label'].values
pixels = df.drop('label', axis=1).values.astype(np.uint8)

max_idx = len(labels) - 1
idx = int(input(f"Enter image index (0 to {max_idx}): "))
if not (0 <= idx <= max_idx):
    raise ValueError(f"Index out of range. Must be between 0 and {max_idx}.")

img_array = pixels[idx].reshape(28, 28)
img = Image.fromarray(img_array, mode='L')
display(img)
print("Label:", labels[idx])
image = pixels[idx].reshape(1, 784)

class Layer:
    def __init__(self, number: int, size: int):
        self._number = number
        self._size = size
        self._data = np.zeros((1, size), dtype=np.float32)

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, new_data):
        if new_data.shape != self._data.shape:
            raise ValueError(f"Expected shape {self._data.shape}, got {new_data.shape}")
        self._data = new_data

class Weight:
    def __init__(self, from_layer: Layer, to_layer: Layer):
        self._value = np.random.randn(from_layer._size, to_layer._size).astype(np.float32)

    @property
    def value(self):
        return self._value

class Bias:
    def __init__(self, to_layer: Layer):
        self._value = np.random.randn(1, to_layer._size).astype(np.float32)

    @property
    def value(self):
        return self._value

class Neural:
    def __init__(self, activation=sigmoid):
        self._activation = activation
        self._layers = []
        self._weights = []
        self._biases = []

    def add_layer(self, layer):
        self._layers.append(layer)
        if len(self._layers) > 1:
            self._weights.append(Weight(self._layers[-2], self._layers[-1]))
            self._biases.append(Bias(self._layers[-1]))

    def forward(self):
        for i in range(1, len(self._layers)):
            prev_data = self._layers[i - 1].data
            weights = self._weights[i - 1].value
            bias = self._biases[i - 1].value
            z = prev_data @ weights + bias
            self._layers[i].data = self._activation(z)

    @property
    def output(self):
        return self._layers[-1].data if self._layers else None

# Build and run network
net = Neural(sigmoid)
input_layer = Layer(1, 784)
input_layer.data = image.astype(np.float32) / 255.0
net.add_layer(input_layer)
net.add_layer(Layer(2, 107))
net.add_layer(Layer(3, 26))
net.add_layer(Layer(4, 10))
net.forward()

# Prediction
predicted_digit = np.argmax(net.output)
print("Predicted Digit:", predicted_digit)

# Cross-Entropy Loss
probs = softmax(net.output)
true_label = labels[idx]
loss = -np.log(probs[0, true_label])
print("Cross-Entropy Loss:", loss)
